## Killariben Limbachiya

# **Modern NLP Approach - Transformer-Based Sentiment Analysis**

# Load The Dataset

In [2]:
# Mount Google Drive (if using Colab)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Load cleaned dataset
input_path = "/content/drive/My Drive/NLP/Assignment_3/fomc_transcripts_spacy_cleaned.csv"
df = pd.read_csv(input_path)
df['Date'] = pd.to_datetime(df['Date'])

Mounted at /content/drive


Install required libraries


In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

In [5]:
import torch
print(torch.__version__)

2.7.0+cu126


In [6]:
!pip install transformers --upgrade --user

In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from collections import Counter  # Import Counter to compute the most frequent label

# Load the FinBERT model from Hugging Face
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a sentiment-analysis pipeline using FinBERT
finbert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


# FinBERT Sentiment Analysis

Sentiment Scoring using FINBERT

In [24]:
!pip install spacy
import spacy
import pandas as pd
from transformers import pipeline, AutoTokenizer
from collections import defaultdict

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Load FinBERT tokenizer and sentiment pipeline with updated arguments
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_pipeline = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer=tokenizer,
    top_k=None  # Return all scores per sentiment
)

# Function to create sliding chunks (~450 tokens per chunk with ~200 token overlap)
def create_sliding_chunks(text, tokenizer, max_tokens=450, stride=200):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    sentence_lengths = [len(tokenizer.encode(sent, add_special_tokens=False)) for sent in sentences]

    chunks = []
    start = 0
    while start < len(sentences):
        token_count = 0
        end = start
        while end < len(sentences) and token_count + sentence_lengths[end] <= max_tokens:
            token_count += sentence_lengths[end]
            end += 1
        chunk = " ".join(sentences[start:end])
        chunks.append(chunk)

        # Move start index forward by approximately stride tokens
        moved = 0
        while start < len(sentences) and moved < stride:
            moved += sentence_lengths[start]
            start += 1
    return chunks

# Function to calculate sentiment score using Positive - Negative formula
def get_sentiment_pos_minus_neg(text, tokenizer, finbert_pipeline):
    chunks = create_sliding_chunks(text, tokenizer)
    sentiment_scores = defaultdict(list)

    for chunk in chunks:
        # Run FinBERT sentiment model on chunk with truncation protection
        scores = finbert_pipeline(chunk, truncation=True, max_length=512)[0]
        for entry in scores:
            label = entry['label'].upper()
            sentiment_scores[label].append(entry['score'])

    # Calculate average scores
    avg_pos = sum(sentiment_scores['POSITIVE']) / len(sentiment_scores['POSITIVE']) if sentiment_scores['POSITIVE'] else 0.0
    avg_neg = sum(sentiment_scores['NEGATIVE']) / len(sentiment_scores['NEGATIVE']) if sentiment_scores['NEGATIVE'] else 0.0

    sentiment_score = round(avg_pos - avg_neg, 4)
    return pd.Series({"sentiment_score": sentiment_score})

# Apply sentiment logic to DataFrame
df[['sentiment_score']] = df['cleaned_text'].apply(
    lambda x: get_sentiment_pos_minus_neg(x, tokenizer, finbert_pipeline)
)



Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1380 > 512). Running this sequence through the model will result in indexing errors


In [26]:
df

,URL,Date,Year,Month,Day,Content,date,GDP,CPI,Unemployment,...,FinBERT_Sentiment,SNR_Ratio_Cleaned,StopWord_Ratio_Cleaned,Redundancy_Ratio_Cleaned,SpecialChar_Density_Cleaned,Semantic_Density_Cleaned,Digit_Ratio_Cleaned,Noise_Score_Cleaned,Quality_Score_Cleaned,sentiment_score
0,https://www.federalreserve.gov/monetarypolicy/...,2025-03-19,2025,2025-03-01,19,HomeMonetary PolicyFederal Open Market Committ...,2025-03-31,-0.3,319.615,4.2,...,None,0.279155,0.354400,0.720845,0.022429,0.408728,0.005838,0.382666,0.305217,-0.4518
1,https://www.federalreserve.gov/monetarypolicy/...,2025-01-29,2025,2025-01-01,29,HomeMonetary PolicyFederal Open Market Committ...,2025-01-31,-0.3,319.086,4.0,...,None,0.273992,0.350740,0.726008,0.022573,0.406120,0.006535,0.379847,0.300264,-0.1370
2,https://www.federalreserve.gov/monetarypolicy/...,2024-12-18,2024,2024-12-01,18,HomeMonetary PolicyFederal Open Market Committ...,2024-12-31,2.4,317.603,4.1,...,None,0.276134,0.353011,0.723866,0.023404,0.410448,0.006984,0.383399,0.303183,-0.3701
3,https://www.federalreserve.gov/monetarypolicy/...,2024-11-07,2024,2024-11-01,7,HomeMonetary PolicyFederal Open Market Committ...,2024-11-30,2.4,316.449,4.2,...,None,0.283448,0.348729,0.716552,0.023190,0.412171,0.005408,0.377328,0.318292,-0.3391
4,https://www.federalreserve.gov/monetarypolicy/...,2024-09-18,2024,2024-09-01,18,HomeMonetary PolicyFederal Open Market Committ...,2024-09-30,3.1,314.851,4.1,...,None,0.271982,0.360516,0.728018,0.022817,0.405509,0.006641,0.389974,0.287517,-0.5391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,https://www.federalreserve.gov/monetarypolicy/...,2015-07-29,2015,2015-07-01,29,HomeMonetary PolicyFederal Open Market Committ...,2015-07-31,1.6,238.034,5.2,...,None,0.245457,0.370972,0.754543,0.019344,0.437740,0.003587,0.393903,0.289294,-0.2055
62,https://www.federalreserve.gov/monetarypolicy/...,2015-06-17,2015,2015-06-01,17,HomeMonetary PolicyFederal Open Market Committ...,2015-06-30,2.5,237.657,5.3,...,None,0.250569,0.369906,0.749431,0.020548,0.426193,0.004593,0.395048,0.281715,-0.1163
63,https://www.federalreserve.gov/monetarypolicy/...,2015-04-29,2015,2015-04-01,29,HomeMonetary PolicyFederal Open Market Committ...,2015-04-30,2.5,236.222,5.4,...,None,0.245321,0.366635,0.754679,0.021659,0.434543,0.004042,0.392337,0.287527,-0.3973
64,https://www.federalreserve.gov/monetarypolicy/...,2015-03-18,2015,2015-03-01,18,HomeMonetary PolicyFederal Open Market Committ...,2015-03-31,3.6,235.976,5.4,...,None,0.246770,0.385646,0.753230,0.019940,0.423773,0.004433,0.410019,0.260525,-0.1754


In [27]:
import pandas as pd
import plotly.graph_objs as go

# Ensure Date is datetime and sorted
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')

# Create a formatted string for Month_Year to use on the x-axis
df['Month_Year'] = df['Date'].dt.strftime('%b_%Y')  # e.g., Jan_2024

# Create line plot of sentiment scores
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Month_Year'],
    y=df['sentiment_score'],
    mode='lines+markers+text',
    text=[f"{score:.2f}" for score in df['sentiment_score']],
    textposition="top center",
    name="Sentiment Score",
    line=dict(color='darkblue', width=3),
    marker=dict(size=8)
))

# Update layout
fig.update_layout(
    title="Sentiment Score Over Time",
    xaxis_title="Month_Year",
    yaxis_title="Sentiment Score",
    xaxis=dict(
        tickmode='array',
        tickvals=df['Month_Year'],
        ticktext=df['Month_Year'],
        tickangle=45
    ),
    yaxis=dict(range=[-1, 1]),
    height=500,
    width=1000
)

fig.show()


In [28]:
df_1 = df.copy()


In [29]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Work with df_1 instead of df, and avoid overwriting original Date
df_1 = df_1.sort_values(by='Date')  # Sort chronologically
df_1['date_new'] = df_1['Date'].dt.strftime('%b_%Y')  # New column for formatted dates

# Create sentiment trend metrics
df_1['sentiment_delta'] = df_1['sentiment_score'].diff()
df_1['sentiment_volatility'] = df_1['sentiment_score'].rolling(window=3, min_periods=1).std()

# Build subplot layout
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=[
        "Sentiment Score Over Time",
        "Month-over-Month Sentiment Change",
        "3-Month Rolling Sentiment Volatility"
    ]
)

# Plot 1: Sentiment Score
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_score'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" for v in df_1['sentiment_score']],
    textposition="top center",
    name="Sentiment Score",
    line=dict(color='darkblue', width=3)
), row=1, col=1)

# Plot 2: Month-over-Month Change
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_delta'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" if pd.notna(v) else "" for v in df_1['sentiment_delta']],
    textposition="top center",
    name="Sentiment Change",
    line=dict(color='darkgreen')
), row=2, col=1)

# Plot 3: Volatility (Rolling Std Dev)
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_volatility'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" if pd.notna(v) else "" for v in df_1['sentiment_volatility']],
    textposition="top center",
    name="Volatility (3-Month Std Dev)",
    line=dict(color='orange')
), row=3, col=1)

# Final layout settings
fig.update_layout(
    height=1000,
    width=1000,
    title="Sentiment Score, Change, and Volatility Over Time",
    showlegend=False
)

fig.update_xaxes(tickangle=45, tickmode='array', tickvals=df_1['date_new'])

fig.show()



In [30]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sort df_1 and format date
df_1 = df_1.sort_values(by='Date')
df_1['date_new'] = df_1['Date'].dt.strftime('%b_%Y')

# Compute all required metrics
df_1['sentiment_delta'] = df_1['sentiment_score'].diff()
df_1['sentiment_volatility'] = df_1['sentiment_score'].rolling(window=3, min_periods=1).std()
df_1['cumulative_sentiment'] = df_1['sentiment_score'].cumsum()

# ------------------- PLOTS 1–3: Main Trend Metrics -------------------
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=[
        "Sentiment Score Over Time",
        "Month-over-Month Sentiment Change",
        "3-Month Rolling Sentiment Volatility"
    ]
)

# Plot 1: Sentiment Score
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_score'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" for v in df_1['sentiment_score']],
    textposition="top center",
    name="Sentiment Score",
    line=dict(color='darkblue', width=3)
), row=1, col=1)

# Plot 2: MoM Change
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_delta'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" if pd.notna(v) else "" for v in df_1['sentiment_delta']],
    textposition="top center",
    name="Sentiment Change",
    line=dict(color='darkgreen')
), row=2, col=1)

# Plot 3: Volatility
fig.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_volatility'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" if pd.notna(v) else "" for v in df_1['sentiment_volatility']],
    textposition="top center",
    name="Volatility (3-Month Std Dev)",
    line=dict(color='orange')
), row=3, col=1)

fig.update_layout(
    height=1000,
    width=1000,
    title="Sentiment Score, Change, and Volatility Over Time",
    showlegend=False
)

fig.update_xaxes(tickangle=45, tickmode='array', tickvals=df_1['date_new'])
fig.show()

# ------------------- PLOT 4: Cumulative Sentiment -------------------
fig_cumulative = go.Figure()
fig_cumulative.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['cumulative_sentiment'],
    mode='lines+markers+text',
    text=[f"{v:.2f}" for v in df_1['cumulative_sentiment']],
    textposition="top center",
    name='Cumulative Sentiment',
    line=dict(color='indigo')
))
fig_cumulative.update_layout(
    title='Cumulative Sentiment Score Over Time',
    xaxis_title='Month_Year',
    yaxis_title='Cumulative Sentiment',
    xaxis=dict(tickangle=45),
    height=500,
    width=1000
)
fig_cumulative.show()

# ------------------- PLOT 5: Histogram of Sentiment Scores -------------------
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(
    x=df_1['sentiment_score'],
    nbinsx=10,
    name='Sentiment Score Distribution',
    marker=dict(color='firebrick')
))
fig_hist.update_layout(
    title="Distribution of Sentiment Scores",
    xaxis_title="Sentiment Score",
    yaxis_title="Frequency",
    height=400,
    width=800
)
fig_hist.show()

# ------------------- PLOT 6: Annotated Min and Max Sentiment -------------------
fig_min_max = go.Figure()
fig_min_max.add_trace(go.Scatter(
    x=df_1['date_new'],
    y=df_1['sentiment_score'],
    mode='lines+markers',
    name='Sentiment Score',
    line=dict(color='slateblue')
))

min_idx = df_1['sentiment_score'].idxmin()
max_idx = df_1['sentiment_score'].idxmax()

fig_min_max.add_trace(go.Scatter(
    x=[df_1.loc[min_idx, 'date_new']],
    y=[df_1.loc[min_idx, 'sentiment_score']],
    text=["Min"],
    mode="markers+text",
    marker=dict(color="red", size=10),
    textposition="bottom center",
    name="Min Score"
))
fig_min_max.add_trace(go.Scatter(
    x=[df_1.loc[max_idx, 'date_new']],
    y=[df_1.loc[max_idx, 'sentiment_score']],
    text=["Max"],
    mode="markers+text",
    marker=dict(color="green", size=10),
    textposition="top center",
    name="Max Score"
))
fig_min_max.update_layout(
    title="Sentiment Score with Min/Max Annotations",
    xaxis_title="Month_Year",
    yaxis_title="Sentiment Score",
    xaxis=dict(tickangle=45),
    height=500,
    width=1000
)
fig_min_max.show()


Generate WordClouds (Yearly)

In [32]:
from wordcloud import WordCloud

years = sorted(df['Year'].unique())
ncols = 3
nrows = (len(years) + ncols - 1) // ncols

plt.figure(figsize=(20, 5 * nrows))

for i, year in enumerate(years):
    year_text = ' '.join(df[df['Year'] == year]['cleaned_text'].astype(str))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(year_text)

    plt.subplot(nrows, ncols, i + 1)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'WordCloud - {year}')

plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.